In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("ultrafeedback_prediction.csv")
df.tail()

,id,winner_model_a,winner_model_b,winner_tie
77043,77043,0.604878,0.134967,0.260155
77044,77044,0.875404,0.045675,0.078920
77045,77045,0.124752,0.685024,0.190224
77046,77046,0.085009,0.745930,0.169061
77047,77047,0.419695,0.192150,0.388155


In [3]:
ultra_win = pd.read_csv("../extra/kaggle-ultrafeedback-drop-duplicate.csv")
ultra_tie = pd.read_csv("../extra/kaggle-ultrafeedback-ties-drop-duplicate.csv")
ultra = pd.concat([ultra_tie, ultra_win], ignore_index=True)
print(ultra.shape)
ultra.head()

(77048, 9)


,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"["" Given a sentence in the English language, t...",starchat,ultralm-65b,0.0,0,0,1,"[""In the Nepali language, the sentence would b...","[""<translation>मोटे मुलुखत प्रतियोगितय अर्थशास..."
1,"["" You are given a background paragraph that d...",llama-2-70b-chat,vicuna-33b,0.0,0,0,1,"[""Output: David is a personal trainer. He has ...","[""David is a marathon runner. One day, he star..."
2,"[""!Hi Chat, I want to develop a python script ...",gpt-3.5-turbo,gpt-4,0.0,0,0,1,"[""Sure! I can help you get started with a Pyth...","[""Here is a Python script that will help you u..."
3,"[""# Exercise: Let's make an Order Dashboard!\n...",bard,gpt-3.5-turbo,0.0,0,0,1,"[""Here are the steps I would take to build the...","[""Thank you for providing the detailed instruc..."
4,"[""###\nLatify helps users on their healthy eat...",llama-2-13b-chat,mpt-30b-chat,0.0,0,0,1,"[""Hello! I'm here to assist you in any way I c...","[""Meno Assist is an app designed to assist wom..."


In [4]:
def find_matching_indices(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("两个列表长度必须相同")
    
    matching_indices = []
    for i in tqdm(range(len(list1)), total=len(list1)):
        if list1[i] == list2[i]:
            matching_indices.append(i)
    
    return matching_indices

In [5]:
threshold = 0.7

predicted_labels = []
gpt4_labels = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    if row["winner_model_a"] >= threshold:
        predicted_labels.append("a")
    elif row["winner_model_b"] >= threshold:
        predicted_labels.append("b")
    elif row["winner_tie"] >= threshold:
        predicted_labels.append("tie")
    else:
        predicted_labels.append("no")

for idx, row in tqdm(ultra.iterrows(), total=len(ultra)):
    if row["winner_model_a"] == 1:
        gpt4_labels.append("a")
    elif row["winner_model_b"] == 1:
        gpt4_labels.append("b")
    elif row["winner_tie"] == 1:
        gpt4_labels.append("tie")

matching_indices = find_matching_indices(predicted_labels, gpt4_labels)
print(len(matching_indices))

100%|██████████| 77048/77048 [00:00<00:00, 3090307.58it/s]

31117


In [6]:
ultra = ultra.iloc[matching_indices]
ultra.reset_index(drop=True, inplace=True)
print(ultra.shape)
ultra.head()

(31117, 9)


,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"["" Given a sentence in the English language, t...",starchat,ultralm-65b,0.0,0,0,1,"[""In the Nepali language, the sentence would b...","[""<translation>मोटे मुलुखत प्रतियोगितय अर्थशास..."
1,"[""$ you are entering a quiet whatsapp conversa...",bard,wizardlm-7b,0.0,0,0,1,"[""...""]","[""...\""""]"
2,"[""(Q).\nThis is a correct answer to the follow...",gpt-3.5-turbo,wizardlm-7b,0.0,0,0,1,"[""No""]","[""No""]"
3,"[""(Q).\n\""Kevork Ajemian\"", given a list of ca...",ultralm-13b,llama-2-70b-chat,0.0,0,0,1,"[""Village""]","[""(A). Village""]"
4,"[""(Question)\nFact 1: Sunlight and rain can ca...",wizardlm-13b,gpt-4,0.0,0,0,1,"[""mutation""]","[""mutations""]"


In [7]:
print(ultra["winner_model_a"].sum())
print(ultra["winner_model_b"].sum())
print(ultra["winner_tie"].sum())

14929
13953
2235
